In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Reshape
from tensorflow.keras.layers import LeakyReLU, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.layers import Conv1DTranspose, Activation, Embedding
from tensorflow.keras.layers import Concatenate, LeakyReLU
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy
import numpy as np

In [2]:
import os
import numpy as np
from tqdm.notebook import tqdm
import guitarpro

import matplotlib.pyplot as plt

from src.markov_chain import MarkovChain
from src.parse_gp import drop_rests_from_drum_track, get_notes_and_durations
from src.save_midi import save_notes_and_durations, save_notes

In [3]:
folderpath = "tabs/"
use_durations = True
track_name = 'guitar'

In [4]:
notes, durations = [], []
for folder in os.listdir(folderpath):
    for filename in tqdm(os.listdir(folderpath + folder + '/')):
        tab = guitarpro.parse(folderpath + folder + '/' + filename)
        tab_notes, tab_durations = get_notes_and_durations(tab, track_name, False)
        notes += tab_notes
        durations += tab_durations

  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
note_dictionary = {note: i for i, note in enumerate(set(notes))}
inv_note_dictionary = {i: note for note, i in note_dictionary.items()}
notes_to_int = [note_dictionary[note] for note in notes]
notes_to_int[:10]

[143, 64, 103, 142, 92, 218, 64, 103, 142, 92]

In [6]:
duration_dictionary = {dur: i for i, dur in enumerate(set(durations))}
inv_dur_dictionary = {i: dur for dur, i in duration_dictionary.items()}
durations_to_int = [duration_dictionary[dur] for dur in durations]
durations_to_int[:10]

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [7]:
def prepare_sequences(notes, durations, length, step, n_notes, n_durations):
    note_sequences = []
    note_targets = []
    dur_sequences = []
    dur_targets = []
    for i in range(0, len(notes) - length, step):
        note_sequences.append(notes[i: i + length])
        note_targets.append(notes[i + length])
        dur_sequences.append(durations[i: i + length])
        dur_targets.append(durations[i + length])
    note_targets = to_categorical(note_targets, n_notes)    
    dur_targets = to_categorical(dur_targets, n_durations)
    sequences = [np.array(note_sequences), np.array(dur_sequences)]
    targets = [note_targets, dur_targets]
    return sequences, targets

In [8]:
X, y = prepare_sequences(notes_to_int, durations_to_int, 128, 4, len(note_dictionary), len(duration_dictionary))
X[0].shape

(1556, 128)

In [67]:
y[0].shape

(1556, 226)

In [90]:
class ModelBuilder(Model):
    def __init__(self, encoder, decoder, loss_factor):
        super(ModelBuilder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.loss_factor = loss_factor

    def compute_loss(self, x, y):
        mu, log_var, z = self.encoder(x)
        generated = self.decoder(z)
        predictions = np.argmax(generated, axis=2)
        entropy = CategoricalCrossentropy(y, generated) * self.loss_factor
        kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mu) - tf.exp(log_var), axis = 1)
        return entropy + kl_loss

    def train_step(self, x, y):
        if isinstance(x, tuple):
            x = x[0]
            y = y[0]
        with tf.GradientTape() as tape:
            loss = self.compute_loss(x, y)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        return {
            "loss": loss
        }
    # ????
    def call(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [91]:
class SequenceVAE:
    
    def __init__(
        self, 
        kernel_size,
        encoder_filters,
        encoder_strides,
        decoder_filters,
        decoder_strides,
        n_notes,
        n_durations,
        emb_size,
        input_dim,
        latent_dim,
        loss_factor
    ):
        self.kernel_size = kernel_size
        self.encoder_filters = encoder_filters
        self.encoder_strides = encoder_strides
        self.decoder_filters = decoder_filters
        self.decoder_strides = decoder_strides
        self.n_notes = n_notes
        self.n_durations = n_durations
        self.emb_size = emb_size
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.loss_factor = loss_factor
        
        self._build_model()
        
        
    def _build_model(self):
        
        # ENCODER
        
        notes_in = Input(shape=(self.input_dim))
        x_notes = Embedding(self.n_notes, self.emb_size)(notes_in)
        durations_in = Input(shape=(self.input_dim))
        x_durations = Embedding(self.n_durations, self.emb_size)(durations_in)
        x = Concatenate()([x_notes, x_durations])
        inp_shape = x.shape[1]
        for i in range(len(self.encoder_filters)):
            x = Conv1D(self.encoder_filters[i], self.kernel_size, self.encoder_strides[i], padding='same')(x)
#             x = BatchNormalization()(x)
            x = LeakyReLU()(x)
        dense_shape = x.shape[1:]
        x = Flatten()(x)
        self.mu = Dense(self.latent_dim)(x)
        self.log_var = Dense(self.latent_dim)(x)
        
        def sample(args):
            mu, log_var = args
            epsilon = K.random_normal(shape=K.shape(mu))
            return mu + K.exp(log_var / 2) * epsilon
        
        self.z = Lambda(sample)([self.mu, self.log_var])
        self.encoder = Model([notes_in, durations_in], [self.mu, self.log_var, self.z])
        
        # DECODER
        
        decoder_input = Input(self.latent_dim)
        x = Dense(np.prod(dense_shape))(decoder_input)
        x = Reshape(dense_shape)(x)
        for i in range(len(self.decoder_filters)):
            x = Conv1DTranspose(self.decoder_filters[i], self.kernel_size, self.decoder_strides[i], padding='same')(x)
#             x = BatchNormalization()(x)
            x = LeakyReLU()(x)

        x_notes, x_durations = tf.unstack(Reshape((2, inp_shape, self.emb_size))(x), axis=1)
        notes_out = Dense(self.n_notes, activation='softmax')(x_notes)
        durations_out = Dense(self.n_durations, activation='softmax')(x_durations)
        self.decoder = Model(decoder_input, [notes_out, durations_out])
        self.model = ModelBuilder(self.encoder, self.decoder, self.loss_factor)
        
    def compile(self, lr):
        self.model.compile(optimizer=Adam(lr))

In [92]:
model = SequenceVAE(
    kernel_size = 32,
    encoder_filters = [128, 128, 64, 32],
    encoder_strides = [1, 2, 2, 1],
    decoder_filters = [16, 32, 64, 128],
    decoder_strides = [1, 2, 2, 1],
    n_notes = len(note_dictionary),
    n_durations = len(duration_dictionary),
    emb_size = 64,
    input_dim = X[0].shape[1],
    latent_dim = 2,
    loss_factor = 1000
)

In [93]:
model.compile(0.001)

In [98]:
model.model.fit(
    X, y,
    shuffle=True,
    batch_size=32,
    epochs=30
)

Epoch 1/30


TypeError: in user code:

    /home/abryl/.cache/pypoetry/virtualenvs/untitled-DiiM6t2A-py3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/abryl/.cache/pypoetry/virtualenvs/untitled-DiiM6t2A-py3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/abryl/.cache/pypoetry/virtualenvs/untitled-DiiM6t2A-py3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/abryl/.cache/pypoetry/virtualenvs/untitled-DiiM6t2A-py3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/abryl/.cache/pypoetry/virtualenvs/untitled-DiiM6t2A-py3.8/li